<a href="https://colab.research.google.com/github/raghus-id/colab/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/raghus-id/colab.git

Cloning into 'colab'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
Receiving objects: 100% (9/9), 5.21 KiB | 5.21 MiB/s, done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Resolving deltas: 100% (1/1), done.


In [ ]:
!pip install pyspark delta-spark ipython-sql sqlalchemy

In [ ]:
from pyspark.sql import SparkSession
from delta import *
builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [ ]:
file_path = "/content/colab/Demographic.csv"

#df = spark.read.option("header",'True').option("inferSchema",'True').csv(file_path)
#df = spark.read.option("header",'True').option("inferSchema",'True').format("csv").load(file_path)
df = spark.read.format("csv").option("header", "true").load(file_path).limit(10)


In [ ]:
delta_df = spark.read.format("csv").option("header", "true").load(file_path).limit(20)

In [ ]:
df = df.withColumnRenamed("Country Name", "Country")\
.withColumnRenamed("Country Code", "Code")\
.withColumnRenamed("Birth rate", "Birth_Rate")\
.withColumnRenamed("Internet users", "Internet_Users")\
.withColumnRenamed("Income Group", "Income_Group")

In [ ]:
delta_df = delta_df.withColumnRenamed("Country Name", "Country")\
.withColumnRenamed("Country Code", "Code")\
.withColumnRenamed("Birth rate", "Birth_Rate")\
.withColumnRenamed("Internet users", "Internet_Users")\
.withColumnRenamed("Income Group", "Income_Group")

In [ ]:
df.write.format("delta").mode("overwrite").saveAsTable("demographic")

In [ ]:
delta_df.write.format("delta").mode("overwrite").saveAsTable("demographic_delta")

In [ ]:
tgt_tab = DeltaTable.forPath(spark, '/content/spark-warehouse/demographic') #df
tgt_tab.toDF().show()

+--------------------+----+----------+--------------+-------------------+
|             Country|Code|Birth_Rate|Internet_Users|       Income_Group|
+--------------------+----+----------+--------------+-------------------+
|               Aruba| ABW|    10.244|          78.9|        High income|
|         Afghanistan| AFG|    35.253|           5.9|         Low income|
|              Angola| AGO|    45.985|          19.1|Upper middle income|
|             Albania| ALB|    12.877|          57.2|Upper middle income|
|United Arab Emirates| ARE|    11.044|            88|        High income|
|           Argentina| ARG|    17.716|          59.9|        High income|
|             Armenia| ARM|    13.308|          41.9|Lower middle income|
| Antigua and Barbuda| ATG|    16.447|          63.4|        High income|
|           Australia| AUS|      13.2|            83|        High income|
|             Austria| AUT|       9.4|       80.6188|        High income|
+--------------------+----+----------+

In [ ]:
src_tab = DeltaTable.forPath(spark, '/content/spark-warehouse/demographic_delta') #delta_df
src_tab.toDF().show()

+--------------------+----+----------+--------------+-------------------+
|             Country|Code|Birth_Rate|Internet_Users|       Income_Group|
+--------------------+----+----------+--------------+-------------------+
|               Aruba| ABW|    10.244|          78.9|        High income|
|         Afghanistan| AFG|    35.253|           5.9|         Low income|
|              Angola| AGO|    45.985|          19.1|Upper middle income|
|             Albania| ALB|    12.877|          57.2|Upper middle income|
|United Arab Emirates| ARE|    11.044|            88|        High income|
|           Argentina| ARG|    17.716|          59.9|        High income|
|             Armenia| ARM|    13.308|          41.9|Lower middle income|
| Antigua and Barbuda| ATG|    16.447|          63.4|        High income|
|           Australia| AUS|      13.2|            83|        High income|
|             Austria| AUT|       9.4|       80.6188|        High income|
|          Azerbaijan| AZE|      18.3|

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
tgt_tab.alias("tgt").merge(delta_df.alias("src"), "tgt.Code = src.Code")\
  .whenMatchedUpdate(condition="tgt.Code = src.Code", set={'tgt.code': lit(None)})\
  .whenNotMatchedInsert(
    values={'tgt.country': 'src.country','tgt.code': lit('delta'), 'tgt.birth_rate': 'src.birth_rate', 'tgt.internet_users': 'src.internet_users', 'tgt.income_group': 'src.income_group'})\
  .execute()

In [ ]:
tgt_tab.toDF().show()

+--------------------+-----+----------+--------------+-------------------+
|             Country| Code|Birth_Rate|Internet_Users|       Income_Group|
+--------------------+-----+----------+--------------+-------------------+
|               Aruba| NULL|    10.244|          78.9|        High income|
|         Afghanistan| NULL|    35.253|           5.9|         Low income|
|              Angola| NULL|    45.985|          19.1|Upper middle income|
|             Albania| NULL|    12.877|          57.2|Upper middle income|
|United Arab Emirates| NULL|    11.044|            88|        High income|
|           Argentina| NULL|    17.716|          59.9|        High income|
|             Armenia| NULL|    13.308|          41.9|Lower middle income|
| Antigua and Barbuda| NULL|    16.447|          63.4|        High income|
|           Australia| NULL|      13.2|            83|        High income|
|             Austria| NULL|       9.4|       80.6188|        High income|
|          Azerbaijan|del

In [ ]:
from google.colab import userdata
print(userdata.get('gituser'))

raghus-id
